In [ ]:


from typing import Dict, Any
from dataclasses import dataclass
from agentic.configs.loader import get_model_config, get_reasoning_config
from agentic.tools.base import BaseTool, ToolMetadata, ToolCategory


@dataclass
class AgentToolConfig:
    """Configuration for agent-as-tool"""
    agent_name: str
    description: str
    parameters_schema: Dict[str, Any]


class SimpleAgentTool(BaseTool):
    """Simple wrapper to use agents as tools with BaseTool compatibility"""
    
    def __init__(self, config: AgentToolConfig, llm_client):
        # Create metadata for BaseTool
        metadata = ToolMetadata(
            name=config.agent_name,
            description=config.description,
            category=ToolCategory.INTELLIGENCE,
            requires_approval=False,
            is_dangerous=False
        )
        super().__init__(metadata)
        
        self.config = config
        self.llm_client = llm_client
        
        # Load config for agent behavior
        self.model_config = get_model_config()
        self.reasoning_config = get_reasoning_config()
    
    def get_parameters_schema(self) -> Dict[str, Any]:
        """Get parameters schema for this agent tool"""
        return self.config.parameters_schema
    
    def execute(self, **kwargs) -> Dict[str, Any]:
        """Execute the agent with provided parameters"""
        try:
            if self.config.agent_name == "debate_agent":
                return self._execute_debate(**kwargs)
            elif self.config.agent_name == "task_planner":
                return self._execute_planner(**kwargs)
            else:
                return {"error": f"Unknown agent: {self.config.agent_name}"}
        except Exception as e:
            return {"error": f"Agent execution failed: {str(e)}"}
    
    def _execute_debate(self, topic: str, context: str, perspectives: list = None) -> Dict[str, Any]:
        """Execute debate analysis with config-driven behavior"""
        if not perspectives:
            perspectives = ["pros", "cons", "technical", "practical"]
        
        # Use reasoning config for confidence and retry behavior
        max_reasoning_steps = self.reasoning_config.get('max_reasoning_steps', 10)
        
        try:
            # This is a simplified implementation
            # In production, you'd use the actual debate agent with LLM
            return {
                "success": True,
                "topic": topic,
                "analysis": {
                    "perspectives": [
                        {
                            "name": p,
                            "arguments": [f"Argument for {p} perspective on {topic}"],
                            "evidence": [f"Evidence supporting {p} view"],
                            "assessment": "neutral"
                        } for p in perspectives
                    ],
                    "recommendation": f"Based on analysis of '{topic}', consider multiple factors before proceeding. Context: {context}",
                    "confidence": 0.7
                },
                "message": "Debate analysis completed (simplified implementation)",
                "model_used": self.model_config.get('name', 'unknown')
            }
        except Exception as e:
            return {"error": f"Debate execution failed: {str(e)}"}
    
    def _execute_planner(self, request: str, complexity: str = "moderate") -> Dict[str, Any]:
        """Execute task planning with config-driven behavior"""
        
        # Use reasoning config for planning depth
        max_reasoning_steps = self.reasoning_config.get('max_reasoning_steps', 10)
        retry_count = self.reasoning_config.get('retry_count', 2)
        
        try:
            # This is a simplified implementation
            # In production, you'd use the actual planner agent with LLM
            
            # Simple task breakdown based on keywords
            tasks = []
            task_count = 0
            
            if "file" in request.lower() or "read" in request.lower():
                task_count += 1
                tasks.append({
                    "id": f"task_{task_count}",
                    "name": "File Operations",
                    "description": "Handle file reading/writing operations",
                    "tools": ["fs_read", "fs_write"],
                    "dependencies": [],
                    "success_criteria": "Files processed successfully"
                })
            
            if "run" in request.lower() or "execute" in request.lower() or "command" in request.lower():
                task_count += 1
                tasks.append({
                    "id": f"task_{task_count}", 
                    "name": "Command Execution",
                    "description": "Execute system commands",
                    "tools": ["execute_bash"],
                    "dependencies": [f"task_{task_count-1}"] if task_count > 1 else [],
                    "success_criteria": "Commands executed without errors"
                })
            
            if "create" in request.lower() or "build" in request.lower():
                task_count += 1
                tasks.append({
                    "id": f"task_{task_count}",
                    "name": "Creation Task",
                    "description": f"Create or build components for: {request}",
                    "tools": ["fs_write", "execute_bash"],
                    "dependencies": [f"task_{i}" for i in range(1, task_count)],
                    "success_criteria": "Components created successfully"
                })
            
            if not tasks:
                tasks.append({
                    "id": "task_1",
                    "name": "General Task",
                    "description": f"Handle request: {request}",
                    "tools": ["fs_read", "fs_write"],
                    "dependencies": [],
                    "success_criteria": "Request completed successfully"
                })
            
            return {
                "success": True,
                "request": request,
                "complexity": complexity,
                "tasks": tasks,
                "execution_order": [task["id"] for task in tasks],
                "estimated_time": f"{len(tasks) * 2}-{len(tasks) * 5} minutes",
                "max_retries": retry_count,
                "message": "Task plan created (simplified implementation)",
                "model_used": self.model_config.get('name', 'unknown')
            }
            
        except Exception as e:
            return {"error": f"Planning execution failed: {str(e)}"}


ImportError: attempted relative import with no known parent package